# Analyze network consequences of protein abundance changes - GPX4
Currently set up for a single model
## Setup
### Import packages

In [ ]:
import io
import re
import shutil
import tempfile
import warnings
import zipfile
from operator import attrgetter
from pathlib import Path
from warnings import warn

import gurobipy as gp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sympy
from cobra.core import get_solution
from cobra.flux_analysis import find_blocked_reactions
from cobra.flux_analysis.variability import flux_variability_analysis
from cobra.manipulation import remove_genes
from rbc_gem_utils import (
    COBRA_CONFIGURATION,
    get_dirpath,
    handle_msg,
    read_cobra_model,
    show_versions,
    write_cobra_model,
)
from rbc_gem_utils.analysis.overlay import (
    DEFAULT_PREFIX_SUFFIX_VALUES,
    DEFAULT_PROTEOME_COMPARTMENT,
    ComplexDilution,
    ProteinDilution,
    add_relaxation_budget,
    load_overlay_model,
    update_slack_value,
)
from rbc_gem_utils.util import AVOGADRO_NUMBER, DEFAULT_DRY_MASS_PER_CELL

gp.setParam("OutputFlag", 0)
gp.setParam("LogToConsole", 0)

# Show versions of notebook
show_versions()

### Define configuration
#### COBRA Configuration

In [ ]:
COBRA_CONFIGURATION.solver = "gurobi"
# Set bound defaults much larger to prevent model loading issues due to protein constraint bounds
COBRA_CONFIGURATION.bounds = (-1e8, 1e8)
COBRA_CONFIGURATION

### Define organism, model, and dataset

In [ ]:
organism = "Human"
model_id = "RBC_GEM"
dataset_name = "DeepRed"

### Set computation options

In [ ]:
protein_of_interest = "GPX4"
ftype = "xml"  # In our experience, SBML/XML loads faster, but will take up to 4x more space uncompressed as compared to JSON
run_computations = True  # Keep off to use previously computed results
overwrite = True  # Whether to allow overwriting of previous simulation results
verbose = True

# Objective reactions
objective_reactions = ["NaKt"]
# Reactions that must have the capability to carry flux, sort for consistency
required_flux_reactions = []  # Add reactions to this list
required_flux_reactions = sorted(set(objective_reactions + required_flux_reactions))

min_relax_budget_for_objectives = True
# Remove blocked reactions before pcFVA simulation.
# For large models and/or multiple runs at different optimums, will speed up computation and potentially improve results.
remove_blocked_reactions = True
# Relaxation reactions that should be restricted to inactive
protein_relaxations_to_restrict = ["GPX1", "GPX4", "PRDX6"]
# Protein constraints that need lower bounds relaxed to prevent increase of
# associated subunits in direct conflict with physiologically known.
# Setting the lower bound prevents from being required.
protein_constraints_lb_to_relax = []

zip_kwargs = dict(compression=zipfile.ZIP_DEFLATED, compresslevel=None)

#### Set prefixes/suffixes to expect

In [ ]:
protein_rxn_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["proteins"]["prefix.dilution"]
protein_met_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["proteins"]["prefix.metabolite"]
relaxation_rxn_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["proteins"]["prefix.relaxation"]
enzyme_met_suffix_total = DEFAULT_PREFIX_SUFFIX_VALUES["enzymes"]["suffix.total"]
enzyme_rxn_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["enzymes"]["prefix.dilution"]
enzyme_met_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["enzymes"]["prefix.metabolite"]
budget_rxn_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["budgets"]["prefix.dilution"]
budget_met_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["budgets"]["prefix.metabolite"]
comp_suffix = f"_{DEFAULT_PROTEOME_COMPARTMENT}"

### Set figure options

In [ ]:
save_figures = True
transparent = False
imagetype = "svg"

### Set paths

In [ ]:
# Set paths
processed_data_dirpath = get_dirpath(use_temp="processed") / organism / dataset_name
# Set paths
overlay_dirpath = get_dirpath("analysis") / "OVERLAY" / organism
model_dirpath = overlay_dirpath / model_id
results_dirpath = (
    get_dirpath(use_temp="processed") / model_id / "OVERLAY" / organism / dataset_name
)
fitting_dirpath = results_dirpath / "fitting"
sample_pcmodels_dirpath = results_dirpath / "pcmodels"
abundance_change_results_path = results_dirpath / "abundance_changes"

# Ensure directories exist
abundance_change_results_path.mkdir(exist_ok=True, parents=True)

## Load RBC-GEM model

In [ ]:
model = read_cobra_model(filename=model_dirpath / f"{model_id}.xml")

ftype = "xml"
with zipfile.ZipFile(f"{sample_pcmodels_dirpath}.zip", "r") as zfile:
    with zfile.open(f"{model_id}_PC_{dataset_name}.{ftype}", "r") as model_file:
        pcmodel = load_overlay_model(
            filename=io.StringIO(model_file.read().decode("utf-8")), filetype=ftype
        )
pcmodel

### Set protein of interest

In [ ]:
protein_gene = pcmodel.genes.get_by_id(protein_of_interest)
protein_met = pcmodel.metabolites.get_by_id(
    f"{protein_met_prefix}{protein_of_interest}{comp_suffix}"
)
protein_rxn = pcmodel.reactions.get_by_id(f"{protein_rxn_prefix}{protein_met.id}")

# Store IDs for later use
protein_mid = protein_met.id
protein_rid = protein_rxn.id
protein_relax_id = f"{relaxation_rxn_prefix}{protein_mid}"

biochemical_reactions = []
for reaction in sorted(protein_gene.reactions, key=attrgetter("id")):
    if reaction in model.reactions:
        print(model.reactions.get_by_id(reaction.id))
        biochemical_reactions += [reaction.id]
required_flux_reactions = list(
    set(required_flux_reactions).union(biochemical_reactions)
)
protein_gene

### Get original values from model

In [ ]:
budget_met_relaxation = pcmodel.metabolites.get_by_id(f"{budget_met_prefix}relaxation")
budget_rxn_relaxation = pcmodel.reactions.get_by_id(
    f"{budget_rxn_prefix}{budget_met_prefix}relaxation"
)

orig_relax_budget = budget_rxn_relaxation.bounds
orig_protein_bounds = protein_rxn.bounds

### Define helper functions

In [ ]:
def find_and_remove_blocked_reactions(
    pcmodel, relaxation_rxns_required=None, prevent_removal=None, verbose=False
):
    pcmodel = pcmodel.copy()
    n_reactions_original = len(pcmodel.reactions)
    n_genes_original = len(pcmodel.genes)
    pcmodel.objective = sum(
        [
            r.flux_expression
            for r in pcmodel.reactions.get_by_any(required_flux_reactions)
        ]
    )
    if prevent_removal is None:
        prevent_removal = []
    for relax_rxn in pcmodel.reactions.query(
        lambda x: x.id.startswith(relaxation_rxn_prefix)
    ):
        if relax_rxn.id in relaxation_rxns_required:
            continue
        else:
            relax_rxn.bounds = (0, 0)
    reactions_to_remove = find_blocked_reactions(
        model=pcmodel,
        reaction_list=None,
        zero_cutoff=COBRA_CONFIGURATION.tolerance,
        open_exchanges=True,
        processes=min(60, COBRA_CONFIGURATION.processes),
    )
    reactions_to_remove = sorted(
        set(reactions_to_remove).difference(
            [
                rxn
                for prot in prevent_removal
                for rxn in [
                    f"{protein_rxn_prefix}{protein_met_prefix}{prot}{comp_suffix}",
                    f"{relaxation_rxn_prefix}{protein_met_prefix}{prot}{comp_suffix}",
                ]
            ]
        )
    )
    pcmodel.remove_reactions(reactions_to_remove, remove_orphans=True)
    genes_to_remove = [
        gene.id
        for gene in pcmodel.genes
        if not pcmodel.reactions.has_id(
            f"{protein_rxn_prefix}{protein_met_prefix}{gene.id}{comp_suffix}"
        )
        and not pcmodel.reactions.has_id(
            f"{relaxation_rxn_prefix}{protein_met_prefix}{gene.id}{comp_suffix}"
        )
    ]
    genes_to_remove = sorted(set(genes_to_remove).difference(prevent_removal))
    remove_genes(pcmodel, gene_list=genes_to_remove, remove_reactions=True)
    handle_msg(
        f"Number of blocked reactions removed: {n_reactions_original - len(pcmodel.reactions)}",
        print_msg=verbose,
    )
    handle_msg(
        f"Number of associated genes removed: {n_genes_original - len(pcmodel.genes)}",
        print_msg=verbose,
    )
    return pcmodel

### Ensure model can optimize all reactions catalyzed by protein of interest

In [ ]:
print("Old relaxation bounds: ({:.6f}, {:.6f})".format(*orig_relax_budget))

# Restrict relaxation reactions for specific proteins
for protein in protein_relaxations_to_restrict:
    protein_met = pcmodel.metabolites.get_by_id(
        f"{protein_met_prefix}{protein}{comp_suffix}"
    )
    relax_prot_rxn = pcmodel.reactions.get_by_id(
        f"{relaxation_rxn_prefix}{protein_met.id}"
    )
    relax_prot_rxn.bounds = (0, 0)
# Relax lower bound constraint for specific proteins
for protein in protein_constraints_lb_to_relax:
    protein_met = pcmodel.metabolites.get_by_id(
        f"{protein_met_prefix}{protein}{comp_suffix}"
    )
    protein_rxn = pcmodel.reactions.get_by_id(f"{protein_rxn_prefix}{protein_met.id}")
    protein_rxn.lower_bound = 0

budget_rxn_relaxation = pcmodel.reactions.get_by_id(
    f"{budget_rxn_prefix}{budget_met_prefix}relaxation"
)
# Determine smallest allowable relxation budget that allows flux through objectives and set as upper bound
with pcmodel:
    pcmodel.objective = (
        sum(
            [
                r.flux_expression
                for r in pcmodel.reactions.get_by_any(required_flux_reactions)
            ]
        )
        - budget_rxn_relaxation.flux_expression
    )
    pcmodel.objective_direction = "max"
    # Fail loudly, should not occur unless a restricted relaxation proteins are absolutely necessary
    pcmodel.slim_optimize(error_value=None)
    relaxation_rxns_required = get_solution(
        pcmodel,
        reactions=pcmodel.reactions.query(
            lambda x: x.id.startswith(relaxation_rxn_prefix)
        ),
    )
    relaxation_rxns_required = set(
        relaxation_rxns_required.fluxes[
            relaxation_rxns_required.fluxes != 0
        ].index.to_list()
    )
    budget_min = budget_rxn_relaxation.flux
if min_relax_budget_for_objectives:
    budget_rxn_relaxation.upper_bound = budget_min
print("New relaxation bounds: ({:.6f}, {:.6f})".format(*budget_rxn_relaxation.bounds))
with warnings.catch_warnings(action="ignore"):
    pcmodel = find_and_remove_blocked_reactions(
        pcmodel,
        relaxation_rxns_required=relaxation_rxns_required,
        prevent_removal=list(
            set(protein_relaxations_to_restrict)
            .union(protein_constraints_lb_to_relax)
            .union([protein_of_interest])
        ),
        verbose=verbose,
    )

pcmodel.objective = sum(
    [r.flux_expression for r in pcmodel.reactions.get_by_any(required_flux_reactions)]
)

sol = pcmodel.optimize(raise_error=True)

# Display solution for required flux capable reactions
objective_sol = sol.fluxes.loc[required_flux_reactions]
print(f"Optimization objective: {sol.objective_value:.6f}\n{objective_sol}")
# Protein utilized
protein_sol = sol.fluxes.loc[[protein_rid, protein_relax_id]]
print(f"\nProtein utilized: {protein_sol.sum():.6f}\n{protein_sol}")
# Required relaxation budget
relax_sol = sol.fluxes.loc[
    pcmodel.reactions.query(
        lambda x: x.id.startswith(relaxation_rxn_prefix)
        or x.id.endswith(budget_met_relaxation.id)
    ).list_attr("id")
]
relax_sol = relax_sol[relax_sol != 0]
relax_budget_used = relax_sol.loc[budget_rxn_relaxation.id]
print(
    f"\nTotal relaxation budget utilized: {relax_budget_used:.6f} ({relax_budget_used / budget_rxn_relaxation.upper_bound:.4%})\n{relax_sol}"
)
pcmodel

## Optimal fluxes as a function of reduction in protein abundance

In [ ]:
percent_interval = (0, -100)
proteome_rxns_of_interest = [
    protein_rid,
    protein_relax_id,
]
biochemical_rxns_of_interest = [
    "G6PDH2",
    "PGL",
    "GND",
    "RPI",
    "RPE",
    "TKT1",
    "TALA",
    "TKT2",
]

solutions = []
with pcmodel:
    protein_reaction = pcmodel.reactions.get_by_id(protein_rid)
    pcmodel.objective = sum(
        [
            rxn.flux_expression
            for rxn in pcmodel.reactions.get_by_any(required_flux_reactions)
        ]
    )

    for percent in np.linspace(
        *percent_interval, 1 + (percent_interval[0] - percent_interval[1])
    ):
        protein_reaction.bounds = (
            orig_protein_bounds[0] * (1 + percent / 100),
            orig_protein_bounds[1] * (1 + percent / 100),
        )
        sol = pcmodel.optimize()
        sols_of_interest = sol.fluxes.loc[
            biochemical_rxns_of_interest
            + [
                x
                for x in required_flux_reactions
                if x not in biochemical_rxns_of_interest
            ]
            + [x for x in proteome_rxns_of_interest if x not in required_flux_reactions]
        ]
        sols_of_interest.name = percent
        solutions.append(sols_of_interest)

df_solutions = pd.concat(solutions, axis=1)
df_solutions

### Plot results for flux as a function of protein abundance

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4.5))

xmin, xmax = (0, 100)
# Plot GTHP
GTHP_flux = df_solutions.loc[["GTHP"]].sum(axis=0)
# For reduction only, switch percentage sign for downward slope
GTHP_flux.index *= -1
GTHP_flux.plot(
    ax=ax,
    color="red",
    linewidth=2,
    label=f"Flux through GTHP",
)
ax.hlines(GTHP_flux.min(), xmin=xmin, xmax=xmax, color="grey", ls=":")
ax.hlines(GTHP_flux.max(), xmin=xmin, xmax=xmax, color="grey", ls=":")
ax.annotate(
    f"$\\approx{GTHP_flux.max():.3f}$",
    xy=(100, GTHP_flux.max() * 0.98),
    xycoords="data",
    fontsize="large",
)
ax.annotate(
    f"$\\approx{GTHP_flux.min():.3f}$",
    xy=(100, GTHP_flux.min() * 0.9),
    xycoords="data",
    fontsize="large",
)

ax.set_xlim(xmin, xmax)
# ax.set_ylim(0, 1)
ax.xaxis.set_tick_params(labelsize="large")
ax.yaxis.set_tick_params(labelsize="large")
ax.set_xlabel(f"Reduction in {protein_of_interest} abundance (%)", fontsize="x-large")
ax.set_ylabel("Flux (mmol/hr/gDW)", fontsize="xx-large")
ax.annotate(
    "GTHP: 2 GSH + H2O2 $\\longrightarrow$ GSSG + 2 H2O",
    xy=(0.5, 1.03),
    ha="center",
    va="center",
    xycoords="axes fraction",
    fontsize="large",
)
ax.legend(
    bbox_to_anchor=(0.5, 1.3), loc="upper center", frameon=False, fontsize="xx-large"
)


ftype = "svg"
fig.tight_layout()
fig.savefig(
    abundance_change_results_path / f"Reduction_{protein_of_interest}_PanelA.{ftype}",
    transparent=True,
    format=ftype,
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5.5))

# Plot GTHP
GTHP_flux = df_solutions.loc[["GTHP"]].sum(axis=0)
# For reduction only, switch percentage sign for downward slope
GTHP_flux.index *= -1
GTHP_flux.plot(
    ax=ax,
    color="red",
    linewidth=2,
    label=f"Flux through GTHP",
)


# Plot sum of all fluxces
sum_of_fluxes = df_solutions.loc[biochemical_reactions].sum(axis=0)
# For reduction only, switch percentage sign for downward slope
sum_of_fluxes.index *= -1
sum_of_fluxes.plot(
    ax=ax,
    color="purple",
    linewidth=2,
    label=f"Sum of fluxes utilizing {protein_of_interest}",
)

# Plot sum of all fluxces excluding GTHP
sum_of_fluxes_wo_GTHP = df_solutions.loc[
    [x for x in biochemical_reactions if x != "GTHP"]
].sum(axis=0)
# For reduction only, switch percentage sign for downward slope
sum_of_fluxes_wo_GTHP.index *= -1
sum_of_fluxes_wo_GTHP.plot(
    ax=ax,
    color="blue",
    linewidth=2,
    label=f"Sum of fluxes utilizing {protein_of_interest}, excluding GTHP",
)


ax.set_xlim(xmin, xmax)
ax.set_ylim(0, 3.5)
ax.xaxis.set_tick_params(labelsize="large")
ax.yaxis.set_tick_params(labelsize="large")

ax.set_xlabel(f"Reduction in {protein_of_interest} abundance (%)", fontsize="xx-large")
ax.set_ylabel("Flux (mmol/hr/gDW)", fontsize="xx-large")
ax.legend(
    bbox_to_anchor=(-0.1, -0.15), loc="upper left", frameon=False, fontsize="x-large"
)

ftype = "svg"
fig.tight_layout()
fig.savefig(
    abundance_change_results_path / f"Reduction_{protein_of_interest}_PanelB.{ftype}",
    transparent=True,
    format=ftype,
)

In [ ]:
fig, axes = plt.subplots(
    2, 1, figsize=(6, 10), sharex=True, sharey=False, height_ratios=[1, 1.4]
)
(ax1, ax2) = axes.flatten()

xmin, xmax = (0, 100)
# Plot GTHP
GTHP_flux = df_solutions.loc[["GTHP"]].sum(axis=0)
# For reduction only, switch percentage sign for downward slope
GTHP_flux.index *= -1
GTHP_flux.plot(
    ax=ax1,
    color="red",
    linewidth=2,
    label=f"Flux through GTHP",
)
ax1.hlines(GTHP_flux.min(), xmin=xmin, xmax=xmax, color="grey", ls=":")
ax1.hlines(GTHP_flux.max(), xmin=xmin, xmax=xmax, color="grey", ls=":")
ax1.annotate(
    f"$\\approx{GTHP_flux.max():.3f}$",
    xy=(100, GTHP_flux.max() * 0.98),
    xycoords="data",
    fontsize="large",
)
ax1.annotate(
    f"$\\approx{GTHP_flux.min():.3f}$",
    xy=(100, GTHP_flux.min() * 0.9),
    xycoords="data",
    fontsize="large",
)

ax1.annotate(
    "GTHP: 2 GSH + H2O2 $\\longrightarrow$ GSSG + 2 H2O",
    xy=(0.02, 0.02),
    xycoords="axes fraction",
    fontsize="large",
)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(0, 1)
ax1.xaxis.set_tick_params(labelsize="large")
ax1.yaxis.set_tick_params(labelsize="large")
# ax1.set_xlabel(f"Reduction in {protein_of_interest} abundance (%)",fontsize="x-large")
ax1.set_ylabel("Flux (mmol/hr/gDW)", fontsize="xx-large")


# Plot GTHP
GTHP_flux = df_solutions.loc[["GTHP"]].sum(axis=0)
# For reduction only, switch percentage sign for downward slope
GTHP_flux.index *= -1
GTHP_flux.plot(
    ax=ax2,
    color="red",
    linewidth=2,
    label=f"Flux through GTHP",
)


# Plot sum of all fluxces
sum_of_fluxes = df_solutions.loc[biochemical_reactions].sum(axis=0)
# For reduction only, switch percentage sign for downward slope
sum_of_fluxes.index *= -1
sum_of_fluxes.plot(
    ax=ax2,
    color="purple",
    linewidth=2,
    label=f"Sum of fluxes utilizing {protein_of_interest}",
)

# Plot sum of all fluxces excluding GTHP
sum_of_fluxes_wo_GTHP = df_solutions.loc[
    [x for x in biochemical_reactions if x != "GTHP"]
].sum(axis=0)
# For reduction only, switch percentage sign for downward slope
sum_of_fluxes_wo_GTHP.index *= -1
sum_of_fluxes_wo_GTHP.plot(
    ax=ax2,
    color="blue",
    linewidth=2,
    label=f"Sum of fluxes utilizing {protein_of_interest}, excluding GTHP",
)


ax2.set_xlim(xmin, xmax)
ax2.set_ylim(0, 3.5)
ax2.xaxis.set_tick_params(labelsize="large")
ax2.yaxis.set_tick_params(labelsize="large")

ax2.set_xlabel(f"Reduction in {protein_of_interest} abundance (%)", fontsize="xx-large")
ax2.set_ylabel("Flux (mmol/hr/gDW)", fontsize="xx-large")
ax2.legend(
    bbox_to_anchor=(-0.02, -0.12), loc="upper left", frameon=False, fontsize="x-large"
)

ftype = "svg"
fig.tight_layout()
fig.savefig(
    abundance_change_results_path / f"Reduction_{protein_of_interest}.{ftype}",
    transparent=True,
    format=ftype,
)